In [1]:
import json
from osgeo import gdal, ogr
import pdal
import os
from tqdm.notebook import tqdm
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from pprint import pprint
import laspy
import time
import sys
from typing import List, Dict, Tuple
from shapely.geometry import box
from shapely.wkt import dumps
import zipfile

gdal.UseExceptions()

# Changes the current path to find the source files
current_dir = os.getcwd()
while current_dir != os.path.abspath("../src"):
    os.chdir("..")
    current_dir = os.getcwd()

from utils import create_all_folders, Folders, download_file

create_all_folders()

In [2]:
# Define tile size and OVERLAP
# TILE_SIZE = 1920  # Size of each tile
# OVERLAP = 480  # Overlap between tiles
TILE_SIZE = 640  # Size of each tile
OVERLAP = 0  # Overlap between tiles

In [3]:
from utils import create_folder, get_file_base_name, open_json
from data_processing import (
    get_image_path_from_full_annotation_path,
    get_cropping_limits,
)

### Get the cropping limits

In [4]:
from data_processing import (
    find_annots_repartition,
    crop_annots_into_limits,
    annots_coordinates_to_local,
    save_annots_per_image,
    crop_all_rgb_and_chm_images_from_annotations_folder,
    ImageData,
)
from lidar_preprocessing import (
    download_lidar_names_shapefile,
    get_lidar_files_from_image,
    download_and_remove_overlap_geotiles,
    create_full_lidar,
)
from chm import compute_chm

In [5]:
annotations_path = os.path.join(Folders.FULL_ANNOTS.value, "10")

annotations = open_json(annotations_path)

full_image_path_tif = get_image_path_from_full_annotation_path(annotations)

image_data = ImageData(full_image_path_tif)

In [6]:
shapefile_path = download_lidar_names_shapefile()

GEOTILES_OVERLAP = 20
intersection_file_names = get_lidar_files_from_image(
    image_data, shapefile_path, GEOTILES_OVERLAP
)
intersection_file_paths = download_and_remove_overlap_geotiles(
    intersection_file_names, GEOTILES_OVERLAP
)

full_lidar_path, full_lidar_filtered_path = create_full_lidar(
    intersection_file_paths, image_data
)

RESOLUTION = 0.08

resolution = RESOLUTION
full_chm_path = os.path.join(
    Folders.CHM.value,
    f"{round(resolution*100)}cm",
    "unfiltered",
    "full",
    f"{image_data.coord_name}.tif",
)
create_folder(os.path.dirname(full_chm_path))
compute_chm(
    full_lidar_path,
    full_chm_path,
    image_data.width_pixel,
    image_data.height_pixel,
    resolution,
    verbose=True,
)
full_chm_filtered_path = os.path.join(
    Folders.CHM.value,
    f"{round(resolution*100)}cm",
    "filtered",
    "full",
    f"{image_data.coord_name}.tif",
)
create_folder(os.path.dirname(full_chm_filtered_path))
compute_chm(
    full_lidar_path,
    full_chm_filtered_path,
    image_data.width_pixel,
    image_data.height_pixel,
    resolution,
    verbose=True,
)

Download skipped.
There is already a file at '/home/alexandre/Documents/tree-segmentation/data/lidar/geotiles/25GN1_13.LAZ'.
Download skipped.
There is already a file at '/home/alexandre/Documents/tree-segmentation/data/lidar/geotiles/25GN1_18.LAZ'.
The file /home/alexandre/Documents/tree-segmentation/data/chm/8cm/unfiltered/full/122000_484000.laz already exists.
The file /home/alexandre/Documents/tree-segmentation/data/chm/8cm/filtered/full/122000_484000.laz already exists.


In [6]:
cropping_limits_x, cropping_limits_y = get_cropping_limits(
    full_image_path_tif, TILE_SIZE, OVERLAP
)
visibility_threshold = 0.2
annots_repartition = find_annots_repartition(
    cropping_limits_x, cropping_limits_y, annotations, visibility_threshold
)
crop_annots_into_limits(annots_repartition)
annots_coordinates_to_local(annots_repartition)

output_image_prefix = get_file_base_name(full_image_path_tif)
annotations_output_directory = os.path.join(
    Folders.CROPPED_ANNOTS.value, output_image_prefix
)
save_annots_per_image(
    annots_repartition, annotations_output_directory, full_image_path_tif
)

images_output_directory = os.path.join(
    Folders.CROPPED_IMAGES.value, output_image_prefix
)
crop_all_rgb_and_chm_images_from_annotations_folder(
    annotations_output_directory, resolution, full_image_path_tif
)

/home/alexandre/miniforge3/envs/tree-segment/lib/python3.12/site-packages/PIL/Image.py:3218: DecompressionBombWarning: Image size (156250000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [ ]:
# crop_las(
#     "../data/lidar/unfiltered/full/122000_484000.laz",
#     "../data/lidar/unfiltered/full/122000_484000_cropped.laz",
#     (image_data.coord_box.x_min, image_data.coord_box.x_max  - 0.9*(image_data.coord_box.x_max - image_data.coord_box.x_min)),
#     (
#         image_data.coord_box.y_min
#         + 0.9 * (image_data.coord_box.y_max - image_data.coord_box.y_min),
#         image_data.coord_box.y_max,
#     ),
# )